<a href="https://colab.research.google.com/github/Maraudier/MatChatBot/blob/master/SQuADChatBotBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Obtain your Kaggle token. 
token = {"username":"lightshift","key":"b7ab47deb22bd26d7cb071a39c232d1c"}
#Colab and Kaggle set up
from google.colab import files
!pip install kaggle
!mkdir .kaggle
!mkdir ~/.kaggle
import json 
import os
if not os.path.exists('/content/.kaggle/kaggle.json'):
  with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
  !chmod 600 /content/.kaggle/kaggle.json
  !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
  !kaggle config set -n path -v{/content}

  !kaggle datasets download -d stanfordu/stanford-question-answering-dataset -p /content
  !unzip \*.zip
  !rm -rf /content/dataset/dataset/
  !ls

mkdir: cannot create directory ‘.kaggle’: File exists
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
#Set up GPU utility and print confirmation
import tensorflow as tf
import torch
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
  print('GPU confirmed: {}'.format(device_name))
else:
  raise SystemError("No GPU")

if torch.cuda.is_available():
  device = torch.device("cuda")
  print("%d GPU(s) " % torch.cuda.device_count())
  print("Using ", torch.cuda.get_device_name(0))

else:
  print("No GPU, defaulting to CPU")
  device = torch.device("cpu")

GPU confirmed: /device:GPU:0
1 GPU(s) 
Using  Tesla K80


In [0]:
#Obtain inputs
import numpy as np
import pandas as pd
import json
from subprocess import check_output

In [0]:
#data management step 1
squad_df = pd.read_json("train-v1.1.json", orient='columns')
#Only 2 columns, data includes nested json. More work to organize data 
squad_df.head()

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1
3,"{'title': 'Genocide', 'paragraphs': [{'context...",1.1
4,"{'title': 'Antibiotics', 'paragraphs': [{'cont...",1.1


In [0]:
#Define a few functions to sort through the data further 
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'], verbose = 1):
  if verbose:
    print("Reading json...")
  file = json.loads(open(input_file_path).read())
  if verbose:
    print("Working...")
  js = pd.io.json.json_normalize(file , record_path )
  m = pd.io.json.json_normalize(file, record_path[:-1])
  r = pd.io.json.json_normalize(file,record_path[:-2])
  
  idx = np.repeat(r['context'].values, r.qas.str.len())
  ndx = np.repeat(m['id'].values, m['answers'].str.len())
  m['context'] = idx
  js['q_idx'] = ndx
  main =pd.concat([m[['id', 'question' , 'context']].set_index('id'),js.set_index('q_idx')], 1, sort=False).reset_index()
  main['c_id'] = main['context'].factorize()[0]
  if verbose:
    print("dataframe shape: {}".format(main.shape))
    print("Jobs Done")
  return main

In [0]:
input_file_path = 'train-v1.1.json'
record_path = ['data', 'paragraphs', 'qas', 'answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path)

Reading json...
Working...
dataframe shape: (87599, 6)
Jobs Done


In [0]:
train.head()

,index,question,context,answer_start,text,c_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0


In [0]:
!pip install transformers

In [0]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

question = train[['question']].copy()
text = train[['text']].copy()
#input_ids = tokenizer.encode(question, text)
#token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
#start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))

#all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
#answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])

#assert answer == "a nice puppet"

In [0]:
q_and_a = train[['question', 'text']].copy()
q_and_a.head()

,question,text
0,To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,What is in front of the Notre Dame Main Building?,a copper statue of Christ
2,The Basilica of the Sacred heart at Notre Dame...,the Main Building
3,What is the Grotto at Notre Dame?,a Marian place of prayer and reflection
4,What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary


In [0]:
#This Transformers package is a pytorch interface for working with BERT
!pip install transformers

In [0]:
#The Corpus of Linguistic Acceptability (CoLA) contains sets of sentances labled by their grammatical correctness
!pip install wget
import wget

url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

# Download the file (if we haven't already)
if not os.path.exists('./cola_public_1.1.zip'):
    wget.download(url, './cola_public_1.1.zip')

if not os.path.exists('./cola_public_1.1.zip'):
  wget.download(url, './cola_public_1.1.zip')

if not os.path.exists('./cola_public/'):
    !unzip cola_public_1.1.zip

In [0]:
import pandas as pd

# Label is the “acceptibility judgment” (0=unacceptable, 1=acceptable).

cola_df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names = ['sentance_source', 'label', 'label_notes', 'sentence'])
sentences = cola_df.sentence.values
label =  cola_df.label.values


#Visual Education
print("Training samples: {:,}\n".format(cola_df.shape[0]))
print("Example of label 0 = Unacceptable sentances ")
display(cola_df.loc[cola_df.label == 0].sample(5)[['sentence', 'label']])
print("\nExample of label 1 = Acceptable sentances")
display(cola_df.loc[cola_df.label == 1].sample(5)[['sentence', 'label']])

Training samples: 8,551

Example of label 0 = Unacceptable sentances 


,sentence,label
7497,We put.,0
2671,Nora sent at the book to Peter.,0
5434,He's a fool enough to try it.,0
7790,Any boy saw no one.,0
7017,Chris will become talking to colleagues.,0



Example of label 1 = Acceptable sentances


,sentence,label
3722,John tagged the monkey in the forest.,1
6106,The audience laughed.,1
642,Rub the cloth on the baby torn.,1
7224,What the spy became was too friendly with his ...,1
6719,The cottage which Mrs Dashwood accepted was ra...,1


In [0]:
from transformers import BertTokenizer
# bert-base-uncased
# 12-layer, 768-hidden, 12-heads, 110M parameters.
# Trained on lower-cased English text.

# bert-large-uncased
# 24-layer, 1024-hidden, 16-heads, 340M parameters.
# Trained on lower-cased English text.

# bert-base-cased
# 12-layer, 768-hidden, 12-heads, 110M parameters.
# Trained on cased English text.

# bert-large-cased
# 24-layer, 1024-hidden, 16-heads, 340M parameters.
# Trained on cased English text.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [0]:
#Visual Education
print("Original: ", sentences[0])
print("Tokenized: ", tokenizer.tokenize(sentences[0]))
print("Token IDs: ", tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

Original:  Our friends won't buy this analysis, let alone the next one we propose.
Tokenized:  ['our', 'friends', 'won', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.']
Token IDs:  [2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012]


In [0]:
input_ids = []
for each_sent in sentences:
  encoded_sentences = tokenizer.encode(each_sent, add_special_tokens = True) 
  input_ids.append(encoded_sentences)
#Visual Education
print('Original: ', sentences[0])
print('TokenIDs: ', input_ids[0])

Original:  Our friends won't buy this analysis, let alone the next one we propose.
TokenIDs:  [101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102]


In [0]:
#Add Padding. Ensure size is equal across the board for all tensors. 
from keras.preprocessing.sequence import pad_sequences
MAX_LENGTH = max([len(sen) for sen in input_ids])
input_ids = pad_sequences(input_ids, 
                          maxlen=MAX_LENGTH, 
                          dtype="long", 
                          value=0, 
                          truncating="post", 
                          padding="post")

In [0]:
#Atten masks helps ensure padding is recognized by the BERT model to be padding. 
attn_masks = []
for sent in input_ids:
  attn_mask = [int(token_id > 0) for token_id in sent]
  attn_masks.append(attn_mask)

In [0]:
#Train Test Split.
from sklearn.model_selection import train_test_split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
  input_ids, 
  label, 
  random_state=888, 
  test_size=.7)
#same for masks, and blanks because we cant use train, test, spit into 3
train_masks, val_masks, _, _ = train_test_split(
    attn_masks, 
    label, 
    random_state=888, 
    test_size=.7)

In [0]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(val_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(val_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(val_masks)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


Got a little crazy here. Need to go back and revit these areas below in more depth

In [0]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 32
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [0]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained(
    "bert-base-uncased",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False)
model.cuda()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [0]:
# #Some options to fine tune with: 
#     # BertModel
#     # BertForMaskedLM
#     # BertForPreTraining
#     # BertForQuestionAnswering
#     # BertForTokenClassification
#     # BertForNextSentencePrediction
#     # BertForSequenceClassification 

In [0]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, #default was 5e-5
                  eps = 1e-8 
                  )

NameError: ignored

In [0]:
from transformers import get_linear_schedule_with_warmup
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


In [0]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [0]:
import random
seed_val = 38
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
loss_values = []
for epoch_i in range(0, epochs):
  print("|-|")
  print('=Epoch {:}/{:}'.format(epoch_i + 1, epochs))
  print("Training...stand by")

  t0 = time.time()

  total_loss = 0

  model.train()

  for step, batch in enumerate(train_dataloader):
    if step % 40 == 0 and not step == 0:
      elapsed = format_time(time.time() - t0)
      print('Batch {:>5,} of {:>5,}. Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

    b_in_ids = batch[0].to(device)
    b_in_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    model.zero_grad()

    outputs = model(b_in_ids, 
                    token_type_ids=None, 
                    attention_mask=b_in_mask, 
                    labels=b_labels)
    
    loss = outputs[0]
    total_loss += loss.item()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
  avg_train_loss = total_loss / len(train_dataloader)

  loss_values.append(avg_train_loss)

  print("|-|")
  print("Avg train loss: {0:.2f}".format(avg_train_loss))


In [0]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(loss_values, 'b-o')

# Label the plot.
plt.title("Training loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.show()

In [0]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = pd.read_csv("./cola_public/raw/out_of_domain_dev.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

# Create sentence and label lists
sentences = df.sentence.values
labels = df.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []

# For every sentence...
for sent in sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   )
    
    input_ids.append(encoded_sent)

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LENGTH, 
                          dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


In [0]:

print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')



In [0]:
print('Positive samples: %d of %d (%.2f%%)' % (df.label.sum(), len(df.label), (df.label.sum() / len(df.label) * 100.0)))

In [0]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  matthews_set.append(matthews)

In [0]:
matthews_set

In [0]:
# Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('MCC: %.3f' % mcc)


In [0]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


In [0]:
# # Mount Google Drive to this Notebook instance.
# from google.colab import drive
#     drive.mount('/content/drive')
#!cp -r ./model_save/ "./drive/Shared drives/ChrisMcCormick.AI/Blog Posts/BERT Fine-Tuning/"
# Load a trained model and vocabulary that you have fine-tuned
# model = model_class.from_pretrained(output_dir)
# tokenizer = tokenizer_class.from_pretrained(output_dir)

# # Copy the model to the GPU.
# model.to(device)
